1) Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.

In [61]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from env import host, username, password

# Acquire

In [80]:
# Get Connection
# Create a helper function to provide connection url for Codeup database server.

def get_db_url(db_name):
    '''
    This function uses my env file to get the url to access the Codeup database.
    It takes in a string identifying the database I want to connect to.
    '''
    return f"mysql+pymysql://{username}:{password}@{host}/{db_name}"

In [81]:
## Create a generic function that takes in a database name and a query.

def get_data_from_sql(str_db_name, query):
    '''
    This function takes in a string for the name of the database I want to connect to
    and a query to obtain my data from the Codeup server and return a DataFrame.
    '''
    df = pd.read_sql(query, get_db_url(str_db_name))
    return df

In [82]:
# write query
query = """
        SELECT 
            customer_id, 
            monthly_charges, 
            tenure, 
            total_charges
        FROM customers
        WHERE contract_type_id = 3;
        """

df = get_data_from_sql('telco_churn', query)

In [74]:
print(f'My df has {df.shape[0]} rows and {df.shape[1]} columns.')

,customer_id,monthly_charges,tenure,total_charges
0,0002-ORFBO,65.60,9,593.30
1,0020-JDNXP,61.25,34,1993.20
2,0022-TCJCI,62.70,45,2791.50
3,0023-UYUPN,25.20,50,1306.30
4,0036-IHMOT,103.70,55,5656.75


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [84]:
df.isna().any()

customer_id        False
monthly_charges    False
tenure             False
total_charges      False
dtype: bool

In [85]:
# find empty cell row
df[df['total_charges'] == ' ']

,customer_id,monthly_charges,tenure,total_charges
234,1371-DWPAZ,56.05,0,
416,2520-SGTTA,20.00,0,
453,2775-SEFEE,61.90,0,
505,3115-CZMZD,20.25,0,
524,3213-VVOLG,25.35,0,
678,4075-WKNIU,73.35,0,
716,4367-NUYAO,25.75,0,
726,4472-LVYGI,52.55,0,
941,5709-LVOEQ,80.85,0,
1293,7644-OMVMY,19.85,0,


In [86]:
# fillna() doesn't work
df['total_charges'] = df['total_charges'].fillna(0)

In [87]:
df[df['total_charges'] == ' ']

,customer_id,monthly_charges,tenure,total_charges
234,1371-DWPAZ,56.05,0,
416,2520-SGTTA,20.00,0,
453,2775-SEFEE,61.90,0,
505,3115-CZMZD,20.25,0,
524,3213-VVOLG,25.35,0,
678,4075-WKNIU,73.35,0,
716,4367-NUYAO,25.75,0,
726,4472-LVYGI,52.55,0,
941,5709-LVOEQ,80.85,0,
1293,7644-OMVMY,19.85,0,


In [88]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [89]:
df.dtypes

customer_id         object
monthly_charges    float64
tenure               int64
total_charges       object
dtype: object

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


Takeaways:
* total_charges == object.  str? 
* tenure has a minimum value of 0.  keep/drop/modify?
* no nulls.


# Prepare

In [91]:
# So these 'missing' values are causing my column to have an object data type.

df.total_charges.value_counts(dropna=False)

           10
7334.05     2
844.45      2
5682.25     2
1161.75     2
           ..
3260.1      1
1629.2      1
3772.5      1
8277.05     1
7325.1      1
Name: total_charges, Length: 1678, dtype: int64

In [92]:
# Notice what these observations also have in common?

df[df.total_charges == ' ']

,customer_id,monthly_charges,tenure,total_charges
234,1371-DWPAZ,56.05,0,
416,2520-SGTTA,20.00,0,
453,2775-SEFEE,61.90,0,
505,3115-CZMZD,20.25,0,
524,3213-VVOLG,25.35,0,
678,4075-WKNIU,73.35,0,
716,4367-NUYAO,25.75,0,
726,4472-LVYGI,52.55,0,
941,5709-LVOEQ,80.85,0,
1293,7644-OMVMY,19.85,0,


In [93]:
# Filter my dataframe to preserve only observations with total_charges; save to new df2.

df2 = df[df.total_charges != ' ']

In [94]:
# Validate that total_charges all have values.

df2[df2.total_charges == ' ']

,customer_id,monthly_charges,tenure,total_charges


In [95]:
# More validating...

df2.total_charges.value_counts(dropna=True).sort_index()

100.35     1
1004.35    1
1005.7     1
1006.9     1
1008.7     1
          ..
973.95     1
978        1
980.35     1
982.95     1
987.95     1
Name: total_charges, Length: 1677, dtype: int64

In [96]:
# I took care of the string values; now I can take care of the data type.

df2.total_charges.dtype

dtype('O')

In [97]:
df2.total_charges = df2.total_charges.astype(float)

In [98]:
# Validate my data types and non-null values. Now I have 1685 observsations.

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1685 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1685 non-null   object 
 1   monthly_charges  1685 non-null   float64
 2   tenure           1685 non-null   int64  
 3   total_charges    1685 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 65.8+ KB


In [99]:
df.tenure.value_counts().sort_index()

0      10
1       2
2       1
3       3
4       4
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 73, dtype: int64

In [100]:
# Replace any tenures of 0 with 1.

df.tenure = df.tenure.replace(0, 1)


In [101]:
# Validate my tenure count for value 1; I went from two to 12.

df.tenure.value_counts().sort_index()

1      12
2       1
3       3
4       4
5       1
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 72, dtype: int64

In [102]:
# These observations also need total_charges handled. 
# The other tenure 1 observations have same monthly and total charges.

df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,
416,2520-SGTTA,20.00,1,
453,2775-SEFEE,61.90,1,
505,3115-CZMZD,20.25,1,
524,3213-VVOLG,25.35,1,
678,4075-WKNIU,73.35,1,
716,4367-NUYAO,25.75,1,
726,4472-LVYGI,52.55,1,
941,5709-LVOEQ,80.85,1,


In [103]:
# Replace the blank total_charges with the monthly_charge for tenure == 1.

df.total_charges = np.where(df.total_charges==' ', df.monthly_charges, df.total_charges)

In [104]:
# Validate my changes.

df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,56.05
416,2520-SGTTA,20.00,1,20
453,2775-SEFEE,61.90,1,61.9
505,3115-CZMZD,20.25,1,20.25
524,3213-VVOLG,25.35,1,25.35
678,4075-WKNIU,73.35,1,73.35
716,4367-NUYAO,25.75,1,25.75
726,4472-LVYGI,52.55,1,52.55
941,5709-LVOEQ,80.85,1,80.85


In [105]:
df.total_charges.dtype

dtype('O')

In [106]:
# Validate my data type conversion.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [107]:
def wrangle_telco():
    """
    Queries the telco_churn database
    Returns a clean df with four columns:
    customer_id(object), monthly_charges(float), tenure(int), total_charges(float)
    """
    query = """
            SELECT 
                customer_id, 
                monthly_charges, 
                tenure, 
                total_charges
            FROM customers
            WHERE contract_type_id = 3;
            """
    df = get_data_from_sql('telco_churn', query)
    
    # Replace any tenures of 0 with 1
    df.tenure = df.tenure.replace(0, 1)
    
    # Replace the blank total_charges with the monthly_charge for tenure == 1
    df.total_charges = np.where(df.total_charges==' ', df.monthly_charges, df.total_charges) 
    
    # Convert total_charges to a float.
    df.total_charges = df.total_charges.astype(float)
    
    return df

In [108]:
# Validate I can call my function and return what I expect.

telco = wrangle_telco()
telco.head()


,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [109]:
# Disco!

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [110]:
def split_continuous(df):
    '''
    Takes in a df
    Returns train, validate, and test DataFrames
    '''
    # Create train_validate and test datasets
    train_validate, test = train_test_split(df, 
                                        test_size=.2, 
                                        random_state=123)
    # Create train and validate datsets
    train, validate = train_test_split(train_validate, 
                                   test_size=.3, 
                                   random_state=123)

    # Take a look at your split datasets

    print(f'train -> {train.shape}')
    print(f'validate -> {validate.shape}')
    print(f'test -> {test.shape}')
    return train, validate, test

In [111]:
train, validate, test = split_continuous(df)

train -> (949, 4)
validate -> (407, 4)
test -> (339, 4)


In [112]:
train.head()

,customer_id,monthly_charges,tenure,total_charges
1256,7501-IWUNG,73.80,61,4616.05
225,1303-SRDOK,69.05,55,3842.6
662,3967-VQOGC,24.90,67,1680.25
628,3777-XROBG,19.55,58,1079.65
824,5075-JSDKI,24.45,59,1493.1
